
#### Hypothèse :
##### Sur la base de la question posée, et sachant que le format OMOP possède d'autres colonnes, je me suis strictement limité aux colonnes demandées pour la table "Person".

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd

# Chargement des données sources
# Lecture de la table contenant les informations des assurés
assurance_info = pd.read_csv("ir_ben_r.csv")

# Affichage de la structure des données 
print(assurance_info.head())

# Étape 1 : Gestion des données manquantes et dupliquées
#verification de la donnée : les lignes dupliquées, les valeurs manquantes, necessité de verifier la coherence, faite en regardant chaque colonne
assurance_info.isna().sum()
assurance_info.duplicated().sum()

# Étape 2 : Mapping des identifiants des personnes
#Afin d'avoir un identifiant OMOP pour les patients, je crée la table person_mapping, et la colonne person_id en autoincrémentation
person_mapping = assurance_info[['NUM_ENQ']].copy()
person_mapping['person_id'] = range(1, len(person_mapping) + 1)
person_mapping

# Étape 3 : Transformation des données 
omop_person = assurance_info.merge(person_mapping, on='NUM_ENQ')
omop_person['person_id'] = omop_person['person_id']
omop_person['gender_concept_id'] = omop_person['ben_sex_cod']
omop_person['year_of_birth']=omop_person['ben_nai_ann']
omop_person['month_of_birth']=omop_person['ben_nai_moi']
omop_person['person_source_value']=omop_person['NUM_ENQ']
omop_person['location_id']=omop_person['ben_res_dpt']
omop_person['gender_source_value']=omop_person['gender_concept_id'].map({1: 'M', 2: 'F'})

#reorganisation de notre table omop_person
omop_person = omop_person[['person_id', 'gender_concept_id', 'year_of_birth', 'month_of_birth', 'person_source_value', 'location_id', 'gender_source_value']]

#Verification de la coherence des types des colonnes, selon la norme officielle de la methode
print(omop_person.dtypes)

#enregistrement en format csv
omop_person.to_csv("omop_person.csv", index=False)
